In [1]:
import pandas as pd
import xlrd 
import os
from ddf_utils.str import to_concept_id, format_float_digits

In [2]:
from functools import partial

In [3]:
sheet_loader = partial(pd.read_excel, skiprows=2)

In [4]:
number_formatter = partial(format_float_digits, digits=8)

In [5]:
source_file = '../source/bp-stats-review-2020-all-data.xlsx'

In [6]:
sheets = xlrd.open_workbook(source_file, on_demand=True).sheet_names()

In [7]:
sheets

['Contents',
 'Primary Energy Consumption',
 'Primary Energy - Cons by fuel',
 'Primary Energy - Cons capita',
 'Carbon Dioxide Emissions',
 'Oil - Proved reserves',
 'Oil - Proved reserves history',
 'Oil Production - Barrels',
 'Oil Production - Tonnes',
 'Oil Production - Crude Conds',
 'Oil Production - NGLs',
 'Total Liquids - Consumption',
 'Oil Consumption - Barrels',
 'Oil Consumption - Tonnes',
 'Oil Consumption - EJ',
 'Oil - Regional Consumption ',
 'Oil - Spot crude prices',
 'Oil - Crude prices since 1861',
 'Oil - Refinery throughput',
 'Oil - Refining capacity',
 'Oil - Regional refining margins',
 'Oil - Trade movements',
 'Oil - Inter-area movements ',
 'Oil - Trade 2018 - 2019',
 'Gas - Proved reserves',
 'Gas - Proved reserves history ',
 'Gas Production - Bcm',
 'Gas Production - Bcf',
 'Gas Production - EJ',
 'Gas Consumption - Bcm',
 'Gas Consumption - Bcf',
 'Gas Consumption - EJ',
 'Gas - Prices ',
 'Gas - Inter-regional trade',
 'Gas - LNG imports',
 'Gas - LNG

In [8]:
contents = pd.read_excel(source_file, sheet_name='Contents', header=None)

In [9]:
contents

,0
0,bp Statistical Review of World Energy June 2020
1,This workbook contains information presented ...
2,"bp Statistical Review of World Energy, which ..."
3,internet at:
4,http://www.bp.com/statisticalreview
...,...
81,Renewable Energy - Solar (Installed capacity)
82,Renewable Energy - Wind (Installed capacity)
83,Approximate conversion factors
84,Definitions


In [10]:
tabs_to_parse = contents.loc[6:82, 0]

In [11]:
tabs_to_parse

6     Primary Energy: Consumption - Exajoules (from ...
7     Primary Energy: Consumption by fuel type - Exa...
8     Primary Energy: Consumption per capita - Gigaj...
9                  Carbon Dioxide Emissions (from 1965)
10                                 Oil: Proved reserves
                            ...                        
78    Key materials - Rare Earth Production - Reserv...
79            Key materials - Cobalt and Lithium Prices
80    Renewable Energy - Geothermal (Installed capac...
81        Renewable Energy - Solar (Installed capacity)
82        Renewable Energy - Wind  (Installed capacity)
Name: 0, Length: 77, dtype: object

In [12]:
len(sheets[1:-2])

77

In [13]:
tabs = pd.DataFrame({'full_name': tabs_to_parse, 'tab_name': sheets[1:-2]})

In [14]:
tabs

,full_name,tab_name
6,Primary Energy: Consumption - Exajoules (from ...,Primary Energy Consumption
7,Primary Energy: Consumption by fuel type - Exa...,Primary Energy - Cons by fuel
8,Primary Energy: Consumption per capita - Gigaj...,Primary Energy - Cons capita
9,Carbon Dioxide Emissions (from 1965),Carbon Dioxide Emissions
10,Oil: Proved reserves,Oil - Proved reserves
...,...,...
78,Key materials - Rare Earth Production - Reserv...,Cobalt and Lithium - Prices
79,Key materials - Cobalt and Lithium Prices,Geothermal Capacity
80,Renewable Energy - Geothermal (Installed capac...,Solar Capacity
81,Renewable Energy - Solar (Installed capacity),Wind Capacity


In [15]:
tabs.to_csv('tabs.csv', index=False)

In [16]:
# Plan:
# production, consumption: create indicators with country/year/fuel/unit dimension (because they have different units)

In [17]:
def preprocess(data):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    data = data.loc[:'total_world']
    data = data.reset_index()
    return data

In [18]:
def apply_map(x, m):
    """general function to replace value in a Series."""
    if x in m:
        return m[x]
    return x

there are a few formats in the excel, we will create handler for each format

```
indicators with only geo, time dimension:

Primary Energy: Consumption - Mtoe (from 1965),Primary Energy Consumption
Carbon Dioxide Emissions (from 1965),Carbon Dioxide Emissions
Oil: Refinery throughput (from 1980),Oil - Refinery throughput
Oil: Refining capacity (from 1965),Oil - Refining capacity
Electricity generation - TWh (from 1985),Electricity Generation
Primary Energy: Consumption per capita - Gigajoule per capita (from 1965),Primary Energy - Cons capita


geo/time/fuel, indicators with unit: 

Oil: Production - Barrels (from 1965),Oil Production - Barrels
Oil: Production - Tonnes (from 1965),Oil Production - Tonnes
Oil: Consumption - Barrels (from 1965),Oil Consumption - Barrels
Oil: Consumption - Tonnes (from 1965),Oil Consumption - Tonnes
Oil: Consumption - Mtoe (from 1965),OIl Consumption - Mtoe
Oil: Refinery throughput (from 1980),Oil - Refinery throughput
Oil: Refining capacity (from 1965),Oil - Refining capacity
Gas: Proved reserves - Bcm (from 1980),Gas - Proved reserves history 
Gas: Production - Bcm (from 1970),Gas Production - Bcm
Gas: Production - Bcf (from 1970),Gas Production - Bcf
Gas: Production - Mtoe (from 1970),Gas Production - Mtoe
Gas: Consumption - Bcm (from 1965),Gas Consumption - Bcm
Gas: Consumption - Bcf (from 1965),Gas Consumption - Bcf
Gas: Consumption - Mtoe (from 1965),Gas Consumption - Mtoe
Coal: Production - Tonnes (from 1981),Coal Production - Tonnes
Coal: Production - Mtoe (from 1981),Coal Production - Mtoe
Coal: Consumption - Mtoe (from 1965),Coal Consumption - Mtoe
Nuclear Energy - Generation - TWh (from 1965),Nuclear Generation - TWh
Nuclear Energy - Consumption - Mtoe (from 1965),Nuclear Consumption - Mtoe
Hydroelectricity - Generation - TWh (from 1965),Hydro Generation - TWh
Hydroelectricity - Consumption - Mtoe (from 1965),Hydro Consumption - Mtoe
Renewables - Other renewables generation -Twh (from 1965),Other renewables - TWh
Renewables - Other renewables consumption - Mtoe (from 1965),Other renewables - Mtoe
Renewables - Solar generation - TWh (from 1965),Solar Consumption - TWh
Renewables - Solar consumption - Mtoe (from 1965),Solar Consumption - Mtoe
Renewables - Wind generation - TWh (from 1965),Wind Consumption - TWh 
Renewables - Wind consumption - Mtoe (from 1965),Wind Consumption - Mtoe
"Renewables - Geothermal, Biomass and Other generation - TWh  (from 1965)",Geo Biomass Other - TWh
"Renewables - Geothermal, Biomass and Other - Mtoe  (from 1965)",Geo Biomass Other - Mtoe
Renewables - Biofuels production - Kboe/d (from 1990),Biofuels Production - Kboed
Renewables - Biofuels production - Ktoe (from 1990),Biofuels Production - Ktoe

Electricity generation from oil -TWh (from 1985),Elec Gen from Oil
Electricity generation from gas - TWh (from 1985),Elec Gen from Gas
Electricity generation from coal - TWh (from 1985),Elec Gen from Coal
Electricity generation from other - TWh (from 1985),Elec Gen from Other

Production - Reserves by key materials:

Key materials - Cobalt Production - Reserves (from 1995),Cobalt Production-Reserves
Key materials - Lithium Production - Reserves (from 1995),Lithium Production-Reserves
Key materials - Graphite Production - Reserves (from 1995),Graphite Production-Reserves
Key materials - Rare Earth Production - Reserves (from 1995),Rare Earth Production-Reserves
Key materials - Cobalt and Lithium Prices,Cobalt and Lithium - Prices

Regional consumption - by product:

Oil: Regional consumption - by product - Barrels (from 1965),Oil - Regional Consumption 


history prices:

Oil: Spot crude prices ,Oil - Spot crude prices
Oil: Crude prices since 1861,Oil - Crude prices since 1861
Gas: Prices ,Gas - Prices 
Coal: Prices,Coal - Prices


class 4:

Oil: Regional refining margins (from 1992),Oil - Regional refining margins

class 5:

Oil: Trade movements (from 1980),Oil - Trade movements
Oil: Inter-area movements ,Oil - Inter-area movements
Oil: Trade 2016-2017,Oil - Trade 2016- 2017
Gas: Trade movements pipeline,Gas - Trade - pipeline
Gas: Trade movements LNG,Gas - Trade movements LNG
Gas: Trade 2016-2017,Gas - Trade 2016-2017


```

In [19]:
def process_1(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  #TODO: change the year column
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2019.1', '2019.2', '2008-18'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]
    d[ddf_id] = d[ddf_id].map(number_formatter)

    return d.sort_values(by=['geo', 'year'])

In [20]:
tabs.iloc[0]

full_name    Primary Energy: Consumption - Exajoules (from ...
tab_name                            Primary Energy Consumption
Name: 6, dtype: object

In [21]:
data1 = sheet_loader(source_file, sheet_name='Primary Energy Consumption')

In [22]:
data1 = preprocess(data1)

In [23]:
data1.head()

,geo,geo_name,1965,1966,1967,1968,1969,1970,1971,1972,...,2015,2016,2017,2018,2019,2019.1,2008-18,2019.2,Unnamed: 59,Unnamed: 60
0,canada,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,...,13.993650,13.936678,14.109482,14.349534,14.214058,-0.009441,0.006404,0.024343,NaN,NaN
1,mexico,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,...,7.688300,7.785419,7.895341,7.833322,7.720742,-0.014372,0.009057,0.013223,NaN,NaN
2,us,US,52.433795,55.383038,57.311776,60.799094,64.049612,66.215508,67.591944,71.064330,...,92.149004,92.017799,92.330525,95.602919,94.648804,-0.009980,0.001054,0.162098,NaN,NaN
3,total_north_america,Total North America,58.408365,61.733704,63.924460,67.868147,71.589254,74.284923,75.943920,80.093438,...,113.830955,113.739897,114.335348,117.785775,116.583604,-0.010206,0.002204,0.199664,NaN,NaN
4,argentina,Argentina,1.127632,1.165108,1.199725,1.241234,1.283912,1.212762,1.288378,1.311442,...,3.585803,3.576195,3.566920,3.540866,3.462769,-0.022056,0.012502,0.005930,NaN,NaN


In [24]:
process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe')).head(100)

,geo,year,primary_energy_consumption_mtoe
3520,algeria,1965,0.08913947
3521,algeria,1966,0.10716806
3522,algeria,1967,0.10142615
3523,algeria,1968,0.10887872
3524,algeria,1969,0.12050818
...,...,...,...
260,argentina,2005,2.802798
261,argentina,2006,2.94482844
262,argentina,2007,3.07479095
263,argentina,2008,3.12715637


In [25]:
df = process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe'))

In [26]:
data1['geo_name'].unique()

array(['Canada', 'Mexico', 'US', 'Total North America', 'Argentina',
       'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru',
       'Trinidad & Tobago', 'Venezuela', 'Central America',
       'Other Caribbean', 'Other South America',
       'Total S. & Cent. America', 'Austria', 'Belgium', 'Bulgaria',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
       'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom',
       'Other Europe', 'Total Europe', 'Azerbaijan', 'Belarus',
       'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'USSR',
       'Uzbekistan', 'Other CIS', 'Total CIS', 'Iran', 'Iraq', 'Israel',
       'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates',
       'Other Middle

In [27]:
# df.to_csv('../../ddf--datapoints--primary_energy_consumption_mtoe--by--geo--year.csv', index=False)

In [28]:
countries = []

In [29]:
tabs_indicator_mapping1 = {
    'Primary Energy Consumption': to_concept_id('Primary Energy Consumption'),
    'Primary Energy - Cons capita': to_concept_id('Primary Energy Consumption per capita'),
    'Carbon Dioxide Emissions': to_concept_id('Carbon Dioxide Emissions'),
    'Oil - Refinery throughput': to_concept_id('Oil - Refinery throughput'),
    'Oil - Refining capacity': to_concept_id('Oil - Refining capacity'),
    'Oil - Proved reserves history': to_concept_id('Oil - Proved reserves'),
    'Gas - Proved reserves history ': to_concept_id('Gas - Proved reserves'),
    'Electricity Generation ': to_concept_id('Electricity Generation')
}

In [30]:
for t, indicator in tabs_indicator_mapping1.items():
    data = sheet_loader(source_file, sheet_name=t)
    data = preprocess(data)
    countries.append(data['geo_name'].unique())
    df = process_1(data, indicator)
    df.to_csv('../../ddf--datapoints--{}--by--geo--year.csv'.format(indicator), index=False)

In [31]:
def make_dict1(sheet, unit, fuel):
    return dict(sheet=sheet, unit=unit, fuel=fuel)

In [32]:
#Mtoe unit is no longer used. Its in Exajoules

tabs_indicator_mapping2 = {
    'production': [
        make_dict1(sheet='Gas Production - Bcm', unit='bcm', fuel='gas'),
        make_dict1(sheet='Gas Production - Bcf', unit='bcf', fuel='gas'),
        # make_dict1('Gas Production - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Production - EJ', 'exajoules', 'gas'),
        make_dict1('Coal Production - Tonnes', 'tonne', 'coal'),
        # make_dict1('Coal Production - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Production - EJ', 'exajoules', 'coal'),
        
        #The Bio Fuels sheet has multiple tables in their sheet like BioDiesel, BioGasoline so they we created
        #another ETL biofuels_run.ipynb for it.
        
        #make_dict1('Biofuels Production - Kboed', 'kboed', 'biofuel'),
        #make_dict1('Biofuels Production - Ktoe', 'ktoe', 'biofuel'),
        make_dict1('Oil Production - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Production - Tonnes', 'tonne', 'oil')
    ],
    'consumption': [
        make_dict1('Gas Consumption - Bcm', 'bcm', 'gas'),
        make_dict1('Gas Consumption - Bcf', 'bcf', 'gas'),
        #make_dict1('Gas Consumption - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Consumption - EJ', 'exajoules', 'gas'),
        #make_dict1('Coal Consumption - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Consumption - EJ', 'exajoules', 'coal'),
        #make_dict1('Nuclear Consumption - Mtoe', 'mtoe', 'nuclear'),
        make_dict1('Nuclear Consumption - EJ', 'exajoules', 'nuclear'),
        #make_dict1('Hydro Consumption - Mtoe', 'mtoe', 'hydro'),
        make_dict1('Hydro Consumption - EJ', 'exajoules', 'hydro'),
        # make_dict1('Other renewables - Mtoe', 'mtoe', 'other_renewables'),
        #make_dict1('Solar Consumption - Mtoe', 'mtoe', 'solar'),
        make_dict1('Solar Consumption - EJ', 'exajoules', 'solar'),
        #make_dict1('Wind Consumption - Mtoe', 'mtoe', 'wind'),
        make_dict1('Wind Consumption - EJ', 'exajoules', 'wind'),
        #make_dict1('Geo Biomass Other - Mtoe', 'mtoe', 'geo_biomass_other'),
        make_dict1('Geo Biomass Other - EJ', 'exajoules', 'geo_biomass_other'),
        make_dict1('Oil Consumption - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Consumption - Tonnes', 'tonne', 'oil'),
        #make_dict1('Oil Consumption - Mtoe', 'mtoe', 'oil')
        make_dict1('Oil Consumption - EJ', 'exajoules', 'oil')
    ],
    'electricity_generation':[
        make_dict1('Elec Gen from Oil', 'twh', 'oil'),
        make_dict1('Elec Gen from Gas', 'twh', 'gas'),
        make_dict1('Elec Gen from Coal', 'twh', 'coal'),
        make_dict1('Elec Gen from Other', 'twh', 'other'),
        make_dict1('Nuclear Generation - TWh', 'twh', 'nuclear'),
        make_dict1('Hydro Generation - TWh', 'twh', 'hydro'),
        # make_dict1('Other renewables - TWh', 'twh', 'other_renewables'),
        make_dict1('Solar Generation - TWh', 'twh', 'solar'),
        make_dict1('Wind Generation -TWh', 'twh', 'wind'),
        make_dict1('Geo Biomass Other - TWh', 'twh', 'geo_biomass_other')
    ]
}

In [33]:
for i, ms in tabs_indicator_mapping2.items():
    #Hisham: I made this datapoint indicator--by--geo--year.csv
    #Previously it was datapoints--indicator--by--geo--fuel--year--unit.csv
    for m in ms:
        data = []
        d = sheet_loader(source_file, sheet_name=m['sheet'])
        d = preprocess(d)        
        countries.append(d['geo_name'].unique())
        indicator_name = m['fuel'] + '_' + i + '_' + m['unit']
        d = process_1(d, indicator_name)        
        #d['fuel'] = m['fuel']
        #d['unit'] = m['unit']
        data.append(d)
        data = pd.concat(data, ignore_index=True)
        df = data[['geo', 'year', indicator_name]]    
        df[indicator_name] = df[indicator_name].map(number_formatter)
        df = df.sort_values(by=['geo', 'year'])
        
        df.to_csv('../../ddf--datapoints--{}--by--geo--year.csv'.format(m['fuel'] + '_' + i + '_' + m['unit']), index=False)
        
    '''data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'fuel', 'year', 'unit', i]]    
    df[i] = df[i].map(number_formatter)
    df = df.sort_values(by=['fuel', 'unit', 'geo', 'year'])
    print(df.head(100))
    df.to_csv('../../ddf--datapoints--{}--by--geo--fuel--year--unit.csv'.format(i), index=False)'''

In [34]:
import numpy as np

In [35]:
countries = np.concatenate(countries)

In [36]:
countries

array(['Canada', 'Mexico', 'US', ..., 'Other Asia Pacific',
       'Total Asia Pacific', 'Total World'], dtype=object)

In [37]:
c1 = pd.DataFrame({'name': countries})

In [38]:
c1['name'] = c1['name'].str.strip()

In [39]:
c1 = c1.drop_duplicates(subset='name')

In [40]:
c1['geo'] = c1['name'].map(to_concept_id)

In [41]:
c1 = c1[['geo', 'name']].sort_values(by='geo')

In [42]:
c1.head()

,geo,name
70,algeria,Algeria
516,angola,Angola
4,argentina,Argentina
80,australia,Australia
16,austria,Austria


In [43]:
c1[c1['name'] == 'North Macedonia']

,geo,name
36,north_macedonia,North Macedonia


In [44]:
!open ../../

/bin/bash: open: command not found


In [45]:
# Reserves by key materials

In [46]:
def preprocess_2(data):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    if 'world' in data.index:
        data = data.loc[:'world']
    else:
        data = data.loc[:'total_world']
    data = data.reset_index()
    return data

def process_2(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  # TODO
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2017.1', '2017.2', '2006-16'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]

    return d.sort_values(by=['geo', 'year'])

In [47]:
countries2 = []

In [48]:
def create_datapoint_2(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries2.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['material'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'material', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    # df.to_csv('../../ddf--datapoints--{}--by--geo--mineral--year.csv'.format(indicator), index=False)
    
    return df

In [49]:
data_tabs = ['Cobalt Production-Reserves', 
             'Lithium Production-Reserves',
             'Graphite Production-Reserves',
             'Rare Earth Production-Reserves'
            ]
fuels  = ['cobalt', 'lithium', 'graphite', 'rare_earth']
indicator = 'production_reserve'

In [50]:
df = create_datapoint_2(data_tabs, fuels, indicator)

In [51]:
countries2 = np.concatenate(countries2)

In [52]:
c2 = pd.DataFrame({'name': countries2})

In [53]:
c2['name'] = c2['name'].str.strip()

In [54]:
c2 = c2.drop_duplicates(subset='name')

In [55]:
c2['geo'] = c2['name'].map(to_concept_id)

In [56]:
c2[~c2.geo.isin(c1.geo)]

,name,geo
2,DR Congo,dr_congo
3,Cuba,cuba
4,Madagascar,madagascar
6,New Caledonia,new_caledonia
11,Zambia,zambia
12,Rest of World,rest_of_world
24,Brazil1,brazil1
27,India2,india2
30,Mozambique,mozambique
35,Rest of World3,rest_of_world3


In [57]:
df.loc[df.geo == 'brazil1', 'geo'] = 'brazil'
df.loc[df.geo == 'india2', 'geo'] = 'india'
df.loc[df.geo == 'rest_of_world2', 'geo'] = 'rest_of_world'
df.loc[df.geo == 'rest_of_world3', 'geo'] = 'rest_of_world'

In [58]:
df.loc[df.geo == 'world', 'geo'] = 'total_world'

In [59]:
# df = df[~df.geo.isin(['rest_of_world'])]

In [60]:
df.sample(10)

,geo,material,year,production_reserve
785,sri_lanka,graphite,2007,9.593
534,us,lithium,2014,0.9
927,china,rare_earth,2015,105
1056,total_world,rare_earth,2017,129.815
624,china,graphite,2004,700
76,dr_congo,cobalt,1996,2
376,brazil,lithium,2004,0.242
274,total_world,cobalt,2002,44.904
589,brazil,graphite,2019,96
718,mozambique,graphite,1998,5.889


In [61]:
df.geo.unique() 

array(['australia', 'canada', 'cuba', 'dr_congo', 'madagascar', 'morocco',
       'new_caledonia', 'papua_new_guinea', 'philippines',
       'rest_of_world', 'russian_federation', 'south_africa',
       'total_world', 'zambia', 'argentina', 'brazil', 'chile', 'china',
       'portugal', 'us', 'zimbabwe', 'india', 'mexico', 'mozambique',
       'sri_lanka', 'ukraine', 'malaysia', 'thailand'], dtype=object)

In [62]:
(df.sort_values(by=['material', 'geo', 'year'])
 .to_csv('../../ddf--datapoints--production_reserve--by--geo--material--year.csv', index=False))

In [63]:
sheet_loader2 = partial(pd.read_excel, skiprows=3)

In [64]:
countries3 = []

In [65]:
def create_datapoint_3(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader2(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries3.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['renewable'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'renewable', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    # df.to_csv('../../ddf--datapoints--{}--by--geo--mineral--year.csv'.format(indicator), index=False)
    
    return df

In [66]:
data_tabs = ['Geothermal Capacity', 
             'Solar Capacity',
             'Wind Capacity'
            ]
fuels  = ['geothermal', 'solar', 'wind']
indicator = 'installed_capacity'

In [67]:
df = create_datapoint_3(data_tabs, fuels, indicator)

In [68]:
df.sample(10)

,geo,renewable,year,installed_capacity
2999,romania,wind,2012,1822
3431,united_kingdom,wind,2014,13074
3104,sweden,wind,2005,522
2353,costa_rica,wind,2004,68.6
2728,new_zealand,wind,2014,682
2537,iran,wind,2004,24.9
354,new_zealand,geothermal,2002,373
3126,taiwan,wind,2004,8.5
1178,india,solar,2019,35059.884
2526,india,wind,2016,28700.44


In [69]:
c3 = np.concatenate(countries3)

In [70]:
c3 = pd.DataFrame({'name': c3})

In [71]:
c3['name'] = c3['name'].str.strip()

In [72]:
c3 = c3.drop_duplicates(subset='name')
c3['geo'] = c3['name'].map(to_concept_id)

In [73]:
print(c3[~c3.geo.isin(c1.geo)][['geo', 'name']].to_csv(index=False))

geo,name
costa_rica,Costa Rica
el_salvador,El Salvador
guatemala,Guatemala
honduras,Honduras
nicaragua,Nicaragua
other_s_and_cent_america,Other S. and Cent. America
ethiopia,Ethiopia
kenya,Kenya
jordan,Jordan
uruguay,Uruguay



In [74]:
df.loc[df.geo == 'russian_fed', 'geo'] = 'russian_federation'

In [75]:
(df.sort_values(by=['renewable', 'geo', 'year'])
 .to_csv('../../ddf--datapoints--{}--by--geo--renewable--year.csv'.format(indicator), index=False))

In [76]:
m2 = {'brazil1': 'brazil', 'india2': 'india', 
      'rest_of_world2': 'rest_of_world', 
      'rest_of_world3': 'rest_of_world'}
c2['geo'] = c2['geo'].map(partial(apply_map, m=m2))

In [77]:
m3 = {'russian_fed': 'russian_federation'}
c3['geo'] = c3['geo'].map(partial(apply_map, m=m3))

In [78]:
geo_ent = pd.concat([c1, c2, c3], ignore_index=True, sort=True)

In [79]:
geo_ent[geo_ent['name'] == 'North Macedonia']

,geo,name
62,north_macedonia,North Macedonia


In [80]:
(geo_ent
 .drop_duplicates(subset='geo')
 .sort_values(by='name')
 .to_csv('../../ddf--entities--geo.csv', index=False))

In [81]:
units = ['Bcm', 'Bcf', 'Exajoules', 'Barrel', 'Tonne', 'TWh', 'Kboed', 'Petajoules']

In [82]:
units_df = pd.DataFrame({'unit': list(map(to_concept_id, units)), 'name': units})

In [83]:
units_df

,unit,name
0,bcm,Bcm
1,bcf,Bcf
2,exajoules,Exajoules
3,barrel,Barrel
4,tonne,Tonne
5,twh,TWh
6,kboed,Kboed
7,petajoules,Petajoules


In [84]:
units_df.to_csv('../../ddf--entities--unit.csv', index=False)

In [85]:
# 

In [86]:
!cp fuel.csv ../../ddf--entities--fuel.csv

In [87]:
materials = pd.DataFrame({'material': ['cobalt', 'lithium', 'graphite', 'rare_earth'], 
                     'name': ['Cobalt', 'Lithium', 'Graphite', 'Rare Earth']})

In [88]:
materials.to_csv('../../ddf--entities--material.csv', index=False)

In [89]:
conc1 = []
conc1_name = []

for n, i in tabs_indicator_mapping1.items():
    conc1.append(i)
    conc1_name.append(n)

In [90]:
concs2 = ['fuel_production', 'fuel_consumption', 'electricity_generation', 'production_reserve', 'installed_capacity']
concs2_name = ['Fuel Production', 'Fuel Consumption', 'Electricity Generation', 'Production Reserve', 'Installed Capacity']

In [91]:
concs = [*conc1, *concs2]
concs_name = [*conc1_name, *concs2_name]

In [92]:
concs

['primary_energy_consumption',
 'primary_energy_consumption_per_capita',
 'carbon_dioxide_emissions',
 'oil_refinery_throughput',
 'oil_refining_capacity',
 'oil_proved_reserves',
 'gas_proved_reserves',
 'electricity_generation',
 'fuel_production',
 'fuel_consumption',
 'electricity_generation',
 'production_reserve',
 'installed_capacity']

In [93]:
measures = pd.DataFrame({'concept': concs, 'name': concs_name})

In [94]:
measures['concept_type'] = 'measure'

In [95]:
measures = measures.drop_duplicates(subset='concept')

In [96]:
measures.to_csv('../../ddf--concepts--continuous.csv', index=False)

In [97]:
disc = pd.DataFrame([
    ['name', 'Name', 'string', ''],
    ['year', 'Year', 'time', ''],
    ['geo', 'Geo', 'entity_domain', ''],
    ['unit', 'Unit', 'entity_domain', ''],
    ['fuel', 'Fuel', 'entity_domain', ''],
    ['renewable', 'Renewables', 'entity_set', 'fuel'],
    ['material', 'Material', 'entity_domain', ''],
    ['domain', 'Domain', 'string', '']
], columns=['concept', 'name', 'concept_type', 'domain'])

In [98]:
disc

,concept,name,concept_type,domain
0,name,Name,string,
1,year,Year,time,
2,geo,Geo,entity_domain,
3,unit,Unit,entity_domain,
4,fuel,Fuel,entity_domain,
5,renewable,Renewables,entity_set,fuel
6,material,Material,entity_domain,
7,domain,Domain,string,


In [99]:
disc.to_csv('../../ddf--concepts--discrete.csv', index=False)